# Задание по курсу «Дискретная оптимизация», МФТИ, весна 2017

## Задача 1-1
## Учимся переводить между decision/evaluation/search на примере Bin Packing

**Даны:** 
* $[w_1, \dots , w_k], w_i \in [0, 1]$ — веса грузов  
* $n_{\text{bins}}$ — количество корзин грузоподъемностью 1.

Предполагается, что функция `solve_bp_decision(weights, n_bins)` решает «распознавательный» (decision) вариант задачи bin packing. На вход ей подаётся список весов и число корзин. Функция возвращает булевский ответ на вопрос «можно ли заданные веса раскидать по не более чем `n_bins` контейнерам? 

Напишите содержимое функции `solve_bp_evaluation`, которая возвращает оптимальное число корзин (решает evaluation variant задачи bin packing). 

Затем напишите содержимое функции `solve_bp_search`, которая возвращает список номеров корзин, которые при каком-нибудь оптимальном распределении присваиваются весам из списка `weights` (корзины нумеруются с единицы). 

Каждая из следующих функций должна вызывать предыдущую не более чем полиномиальное число раз.

In [50]:
def solve_bp_decision(weights: list, n_bins: int) -> bool:
    def able_to_pack(weights: list, bin_capacities) -> bool:
        return weights == [] or any( 
            able_to_pack( weights[:-1], bin_capacities[:i] + [capacity - weights[-1]] + bin_capacities[(i + 1):] ) 
            for i, capacity in enumerate(bin_capacities) if capacity >= weights[-1] 
        )

    return able_to_pack( sorted(weights), [1.0] * n_bins )

# To test this, run:
solve_bp_decision([0.8, 0.09, 0.4, 0.7], 2), solve_bp_decision([0.8, 0.09, 0.4, 0.7], 3)

(False, True)

In [130]:
def solve_bp_evaluation(weights: list) -> int:
    # Бинпоиск по ответу
    lower = 0
    upper = len(weights)
    while (upper - lower) > 1:   
        x = lower + (upper - lower) // 2
        if solve_bp_decision(weights, x):
            upper = x
        else:
            lower = x
    return upper        
#   … your code here
#   return min_n_bins
    pass

solve_bp_evaluation([0.8, 0.09, 0.4, 0.7]), solve_bp_evaluation([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95])

(3, 6)

In [135]:
def solve_bp_search(weights: list):
    solution = [-1] * len(weights)
    # Основная идея: если solve_bp_evaluation([x1, x2, x3, ...]) = solve_bp_evaluation([x1+x2, 0, x3...]), 
    # то существует оптимальная укладка, в которой первый и второй грузы лежат в одной корзине. Тогда эти грузы можно 
    # склеить в один с весом x1+x2
    answer = solve_bp_evaluation(weights)
    curr = 0 # текущая корзина 
    for i in range(len(weights)):
        if solution[i] == -1:
            solution[i] = curr + 1 
            for j in range(len(weights)):
                if solution[j] == -1:
                    curr_weight = weights[j]
                    if weights[i] + weights[j] > 1:
                        continue
                    weights_1 = weights.copy()
                    weights_1[i] += weights_1[j]
                    weights_1[j] = 0
                    if solve_bp_evaluation(weights_1) == answer:
                        solution[j] = curr + 1
                        weights = weights_1    
            curr += 1
    return solution       
            
        

#   … your code here
#   return solution # list
    pass
solve_bp_search([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]), solve_bp_search([0.8, 0.09, 0.4, 0.7]),
#solve_bp_search([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.1])

([1, 1, 2, 3, 1, 3, 2, 4, 5, 6], [1, 1, 2, 3])

In [153]:

import random
our_weights = []
for i in range(10):
    our_weights.append(random.random())
    
search = solve_bp_search(our_weights) 
print(our_weights, search, solve_bp_evaluation(our_weights))
flag = True
if solve_bp_evaluation(our_weights) != max(search):
    flag = False
else:
    bins_n = max(search)
    for i in range(bins_n):
        summ = 0
        for j in range(len(search)):
            if search[j] == i + 1:
                summ += our_weights[j]
                if summ > 1:
                    flag = False
if flag:
    print("OK")


[0.007373675545275238, 0.360199023147484, 0.42226438010875356, 0.46269474947372113, 0.610655217444486, 0.8654175483373892, 0.8992832494141025, 0.8411844185988167, 0.5683161637328914, 0.06447212718145268] [1, 1, 2, 1, 3, 4, 5, 6, 2, 1] 6
OK
